In [1]:
import tuxml 
import pandas as pd

df = tuxml.load_dataset()

In [2]:
df[:10]

,X86_LOCAL_APIC,OPENVSWITCH,TEXTSEARCH_FSM,LOCKDEP_SUPPORT,GENERIC_CLOCKEVENTS_MIN_ADJUST,NETFILTER_XT_MATCH_TCPMSS,MPLS,NFC_HCI,STACKTRACE_SUPPORT,GENERIC_CMOS_UPDATE,...,LZO-vmlinux,LZO,LZ4-bzImage,LZ4-vmlinux,LZ4,cid,nbyes,nbno,nbmodule,nbyesmodule
14758,1,0,0,1,1,0,1,0,1,1,...,11008072,8734199,9839568.0,11925896,9638560,30000,1435,10346,697,2132
14759,1,0,0,1,1,0,0,0,1,1,...,6603288,4385061,4838864.0,6923096,4693085,30001,1382,10012,1084,2466
14760,1,0,0,1,1,0,0,0,1,1,...,14475800,12256864,13362640.0,15446872,13214970,30002,1626,10852,0,1626
14761,1,0,0,1,1,0,0,0,1,1,...,10867576,8647251,9302992.0,11388080,9155423,30003,2140,10338,0,2140
14762,1,0,0,1,1,0,0,1,1,1,...,20018040,17796721,19346384.0,21431472,19197696,30004,2651,9827,0,2651
14763,1,0,1,1,1,0,0,0,1,1,...,10412920,8193340,8897488.0,10982576,8748171,30005,1591,10048,839,2430
14764,1,0,0,1,1,0,1,0,1,1,...,11576184,9354542,10048464.0,12133552,9901074,30006,2115,10363,0,2115
14765,1,0,0,1,1,0,0,1,1,1,...,35274032,33029312,35424016.0,37510768,35253064,30007,3301,9177,0,3301
14766,1,0,0,1,1,0,0,0,1,1,...,8409976,6188651,6734800.0,8819888,6587832,30008,1510,10968,0,1510
14767,1,0,0,1,1,0,1,0,1,1,...,11170328,8952807,9663440.0,11748184,9518916,30009,2507,9971,0,2507


In [3]:
df.sort_values(by='vmlinux', ascending=True)[['vmlinux', 'nbno', 'nbyes', 'nbmodule', 'nbyesmodule']][:10]

,vmlinux,nbno,nbyes,nbmodule,nbyesmodule
109097,7304656,12237,241,0,241
58621,7317008,12238,240,0,240
107213,8716424,11068,741,669,1410
99232,10486184,11131,636,711,1347
99777,10504040,11351,721,406,1127
91160,10504144,10831,847,800,1647
109537,10511608,11226,724,528,1252
80771,10524528,11163,857,458,1315
66399,10572520,10731,816,931,1747
96643,10648928,10926,835,717,1552


In [4]:
# df.to_pickle("all_size_withyes.pkl")

In [5]:
size_methods = ["vmlinux", "GZIP-bzImage", "GZIP-vmlinux", "GZIP", "BZIP2-bzImage", 
              "BZIP2-vmlinux", "BZIP2", "LZMA-bzImage", "LZMA-vmlinux", "LZMA", "XZ-bzImage", "XZ-vmlinux", "XZ", 
              "LZO-bzImage", "LZO-vmlinux", "LZO", "LZ4-bzImage", "LZ4-vmlinux", "LZ4"]

In [16]:
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.inspection import partial_dependence
# from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np


def run_regressorML(reg, test_size, size_target="vmlinux"):
    assert(size_target in size_methods)
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=["cid"]).drop(columns=size_methods), df[size_target], test_size=test_size, random_state=42)  
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    dfErrors = pd.DataFrame({"error":(y_pred - y_test).abs(), "% error":((y_pred - y_test) / y_test).abs()*100})
    ftsDep = range(0, 5)
    print(partial_dependence(reg, X_test, ftsDep)) # , method='brute')
    return dfErrors["% error"].describe()


In [ ]:

   
#hyperparams_DT = {
    # "criterion": "mse",
#    "max_depth": 12,
#    "min_samples_leaf": 5,
#    "max_leaf_nodes": None,
#}

#reg_dt = tree.DecisionTreeRegressor(**hyperparams_DT)
#res_dt = run_regressorML(reg_dt, test_size=0.9)
#res_dt

hyperparams_RF = {
    "max_depth": 18,
    "n_estimators": 10,
    "n_jobs" : -1
}

reg_rf = ensemble.RandomForestRegressor(**hyperparams_RF)
res_rf = run_regressorML(reg_rf, test_size=0.99, size_target="vmlinux")
res_rf

#hyperparams_GB = {
#    'learning_rate': 0.1, 
   # 'max_depth': 5, 
#    'n_estimators': 300    
#}

#reg_gb = GradientBoostingRegressor()
#res_gb = run_regressorML(reg_gb, test_size=0.9, size_target="vmlinux")
#res_gb


# run_regressorML(ensemble.AdaBoostRegressor(), test_size=0.98, size_target="vmlinux")

In [8]:
def mk_ftimportances(reg):
    col = df.drop(columns=["cid"]).drop(columns=size_methods).columns
    importanceSeries = pd.Series(reg.feature_importances_, index=col.values)
    importanceSeries[importanceSeries > 0].sort_values(ascending=False)

    ftimportance = pd.Series(reg.feature_importances_, index=col.values)
    return ftimportance[ftimportance > 0].sort_values(ascending=False)

#ft_dt = mk_ftimportances(reg_dt)
#ft_dt.to_csv("feature_importanceDT.csv", header=True)
#ft_dt

ft_rf = mk_ftimportances(reg_rf)
ft_rf.to_csv("feature_importanceRF.csv", header=True)
ft_rf

#ft_gb = mk_ftimportances(reg_gb)
#ft_gb.to_csv("feature_importanceGB.csv", header=True)
#ft_gb

DEBUG_INFO                           3.430822e-01
SENSORS_ACPI_POWER                   4.460333e-02
SERIAL_CONEXANT_DIGICOLOR_CONSOLE    3.773853e-02
SND_FIREWIRE_DIGI00X                 3.251581e-02
nbyes                                2.939849e-02
CLS_U32_PERF                         2.854648e-02
SND_HDA_CODEC_REALTEK                2.696558e-02
MMC_SDHCI_OF_ARASAN                  2.638046e-02
GCOV_PROFILE_ALL                     2.540032e-02
SND_HDA_CORE                         2.120636e-02
SND_SOC_TPA6130A2                    1.806301e-02
CRYPTO_POLY1305                      1.699236e-02
DRM_I915_DEBUG                       1.642625e-02
TPS65010                             1.405784e-02
KASAN_OUTLINE                        1.370030e-02
SND_HDA_CODEC_SIGMATEL               1.321853e-02
SENSORS_LM83                         1.283560e-02
SERIAL_XILINX_PS_UART_CONSOLE        1.189288e-02
FB_ATY                               1.135287e-02
SND_HDA_CODEC_CIRRUS                 1.135054e-02


In [9]:
####### column' indices of feature importances
# TODO: enhance!
if (False):
    col = df.drop(columns=["cid"]).drop(columns=size_methods).columns
    ftIndiceImportanceGB = []
    for ft_name in ft_rf.index:
        i = 0
        for vl in col.values:
            if (ft_name == vl):
                ftIndiceImportanceGB.append(i)
    #            print(ft_name, "at", i)
                continue
            i = i + 1
    ftIndiceImportanceGB

    # TODO: instead of plotting, retrieving the coefficients
    col = df.drop(columns=["cid"]).drop(columns=size_methods).columns
    import matplotlib.pyplot as plt
    # TODO: pair-wise feature interactions
    ftsDep = ftIndiceImportanceGB[:20] # importanceGB[importanceGB > 0].sort_values(ascending=False)[:10].values
    plt.figure(figsize=(20, 20))
    # TODO: X_train is an input! (it works only if X_train is accessible/visible)
    fig, ax = plot_partial_dependence(reg_rf, X_train, ftsDep, feature_names=col.values, grid_resolution=100)
    fig.set_figwidth(8)
    fig.set_figheight(15)
    fig.tight_layout()
    plt.show()

In [10]:
import sklearn
sklearn.__version__

'0.21.2'